In [1]:
#Load model
from pyspark import SparkContext
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
sc = SparkContext(appName="PythonDecisionTreePrediction")
Model_runs = DecisionTreeModel.load(sc, "runs")
Model_wickets = DecisionTreeModel.load(sc, "wickets")

In [2]:
#load mapping json
import json
with open('player_map.json','r') as fd:
    player_map = json.load(fd)

In [21]:
import numpy as np
from random import uniform, shuffle
import csv

def random():
    return round(np.random.random(),2)
#predict runs in an over based on  batsman,nonstrike,bowler,home,averagescore,sr,innings using decision tree classifier
#a - > batsman , b->bowler
def pred_runs(a,b,nonstrike,home,inning):
        try:
            batclus = batsman_clus.filter(regex=a,axis=0)
            if len(batclus)!=1:
                batclus = batsman_clus.filter(regex=player_map[a],axis=0)
            bat2clus = batsman_clus.filter(regex=nonstrike,axis=0)
            if len(bat2clus)!=1:
                bat2clus = batsman_clus.filter(regex=player_map[nonstrike],axis=0)
            bowlclus = bowler_clus.filter(regex=b,axis=0)
            if len(bowlclus)!=1:
                 bowlclus = bowler_clus.filter(regex=player_map[b],axis=0)
        except:
            batclus = batsman_cluster_info[batsman_cluster_info.prediction==0]
            bat2clus = batsman_cluster_info[batsman_cluster_info.prediction==0]
            bowlclus = bowler_cluster_info[bowler_cluster_info.prediction==3]
        return Model_runs.predict([batclus.prediction,bat2clus.prediction,bowlclus.prediction,home,batclus.ave_score,batclus.sr,bowlclus.econ,inning]) 
    
    
    
#predict wickets in an over based on batsman,nonstrike,bowler,home,ave,sr,econ,innings using decision tree classifier              
#a - > batsman , b->bowler
def pred_wicket(a,b,nonstrike,home,inning):
        try:
            batclus = batsman_clus.filter(regex=a,axis=0)
            if len(batclus)!=1:
                batclus = batsman_clus.filter(regex=player_map[a],axis=0)
            bat2clus = batsman_clus.filter(regex=nonstrike,axis=0)
            if len(bat2clus)!=1:
                bat2clus = batsman_clus.filter(regex=player_map[nonstrike],axis=0)
            bowlclus = bowler_clus.filter(regex=b,axis=0)
            if len(bowlclus)!=1:
                 bowlclus = bowler_clus.filter(regex=player_map[b],axis=0)
        except:
            batclus = batsman_cluster_info[batsman_cluster_info.prediction==0]
            bat2clus = batsman_cluster_info[batsman_cluster_info.prediction==0]
            bowlclus = bowler_cluster_info[bowler_cluster_info.prediction==3]
        return Model_wickets.predict([batclus.prediction,bat2clus.prediction,bowlclus.prediction,home,bowlclus.bowl_ave,bowlclus.bowl_sr,bowlclus.econ,inning]) 

#use innings to predict the events in an innings over by over
def innings(batsman, bowlers, file_name,home,number,opponent=0):
    d_batsmen = dict()
    d_bowler = dict()
    with open(file_name+'.csv','w') as csvfile:
        fieldnames = ['batsman','bowler','runs','out']
        writer = csv.DictWriter(csvfile, fieldnames = fieldnames)
        writer.writeheader()
        strike = dict()
        non_strike = dict()

        # Team 1 batting
        strike['name'] = batsman[0]
        non_strike['name'] = batsman[1]
        wicket_count = 0
        wicket = 0
        flag_allout = True
        total = 0
        over_number = 1
        while over_number <= 20 and flag_allout:
            over_run = 0
            over_wicket = 0
            print("Over Number ",over_number)
            print("The Bowler is", bowlers[over_number%len(bowlers)])
            bowler = bowlers[over_number%len(bowlers)]
#             for balls in range(1,7):
            runs = pred_runs(strike['name'], bowlers[over_number%len(bowlers)],non_strike['name'],home,number)
            wicket = pred_wicket(strike['name'], bowlers[over_number%len(bowlers)],non_strike['name'],home,number)            
            if wicket:
                wicket_count = wicket_count + 1
                over_wicket +=1
                writer.writerow({'batsman':strike['name'],'bowler':bowler,'runs':runs,'out':wicket})
                d_bowler[bowler] = d_bowler.get(bowler,0)+1
                print(strike['name']+" got out")
                if wicket_count == 10:
                    print("Innings Complete!")
                    flag_allout = False
                    break

                strike['name'] = batsman[wicket_count + 1]
                print(strike['name'] +" is the man next in")
                #print(total,"/",wicket_count)
            if wicket == 0:    
                writer.writerow({'batsman':strike['name'],'bowler':bowler,'runs':runs,'out':wicket})
                d_batsmen[strike['name']] = d_batsmen.get(strike['name'],0)+runs
                over_run+=runs
                print(strike['name']+" Has scored", runs)
                strike, non_strike = non_strike, strike
            total+=over_run
                
            over_number = over_number  + 1
            print()
            print(total, "/", wicket_count)
            print("OVER STATS")
            print("runs:",over_run,", outs",over_wicket)
            print()
            if number == 1 and total>opponent:
                return total
        print()
        print('BATSMAN STATS')
        print(d_batsmen)
        print('BOWLER STATS')
        print(d_bowler)
        print()
        print('INNINGS OVER')
        return total
    
#some matches don't need all 20 overs, however to simulate all bowlers are required. This function completes bowling order
from random import shuffle
def complete_bowling_order(bowlers):
    over_count = dict()
    for bowler in bowlers:
        over_count[bowler] = over_count.get(bowler,0)+1
    fill_bowler = list()
    for key,value in over_count.items():
        if value < 4:
            fill_bowler.extend([key]*(4-value))
    shuffle(fill_bowler)
    bowlers.extend(fill_bowler[:20-len(bowlers)])
    return bowlers


In [22]:
def make_reg(temp):
    temp=temp.split()
    reg=temp[0][0]+'.*'+temp[-1]
    return reg

In [23]:
import pandas as pd

In [24]:
clusters_pred = pd.read_csv('final_cluster.csv') 
batsman_clus = pd.read_csv('batsman_cluster.csv')
bowler_clus =pd.read_csv('bowler_cluster.csv')
batsman_cluster_info = pd.read_csv('batsman_cluster_info.csv')
bowler_cluster_info =pd.read_csv('bowler_cluster_info.csv')
ipl = pd.read_csv('new.csv')
ipl = ipl.set_index('key')
clusters_pred = clusters_pred.set_index('key')
batsman_clus = batsman_clus.set_index('player_name')
bowler_clus = bowler_clus.set_index('player_name')
for i in range(1,6):
    clusters_pred.iloc[:,i]+=clusters_pred.iloc[:,i-1]

clusters_pred = clusters_pred[~clusters_pred.index.duplicated()]
batsman_clus = batsman_clus[~batsman_clus.index.duplicated()]
bowler_clus = bowler_clus[~bowler_clus.index.duplicated()]

In [25]:
#match predictions 1:win,2:

def match(team_1n,team_2n,a,b,c,d,home):
        batsman_1 = a


        bowlers_1 = b


        batsman_2 = c

        bowlers_2 = d
        
        bowlers_1 = complete_bowling_order(bowlers_1)
        bowlers_2 = complete_bowling_order(bowlers_2)
        
        
        runs1=innings(batsman_1, bowlers_2,team_1n+'innings',home^0,0)
        runs2=innings(batsman_2, bowlers_1,team_2n+'innings',home^1,1,runs1)
        if runs1 == runs2:
            print("Match Tied!")
            return 0
        elif runs1 > runs2:
            print(team_1n, "Has won the match by = ", runs1 - runs2)
            return 1
        elif runs2 > runs1:
#             print(team_2n, "Has won the match by = ", runs2 - runs1)
            return 1
        else:
            return -1

In [26]:
#RCB V MI 2018
#Innings 1 RCB bat
Batsmen1 = ['M Vohra','Q de Kock','BB McCullum','V Kohli','AB de Villiers','C de Grandhomme','Washington Sundar','TG Southee','UT Yadav','YS Chahal','Mohammed Siraj']
# Bowlers = ['JP Duminy','MJ McClenaghan','JJ Bumrah','KH Pandya','HH Pandya','M Markande']
Bowlers2 = ['JP Duminy','MJ McClenaghan','JJ Bumrah','JP Duminy','MJ McClenaghan','KH Pandya','M Markande','KH Pandya','M Markande','HH Pandya','KH Pandya','M Markande','KH Pandya','JJ Bumrah','MJ McClenaghan','HH Pandya','JJ Bumrah','HH Pandya','JJ Bumrah','MJ McClenaghan']
#Innings Simulation
#RCB V MI 2018
#Innings 2 MI bat
Batsmen2 = ['SA Yadav','Ishan Kishan','JP Duminy','RG Sharma','KA Pollard','KH Pandya','HH Pandya','BCJ Cutting','MJ McClenaghan','M Markande','JJ Bumrah']
# Bowlers = ['TG Southee','UT Yadav','Mohammed Siraj','YS Chahal','Washington Sundar','C de Grandhomme']
Bowlers1 = ['TG Southee','UT Yadav','TG Southee','UT Yadav','Mohammed Siraj','UT Yadav','YS Chahal','Mohammed Siraj','YS Chahal','Washington Sundar','YS Chahal','C de Grandhomme','YS Chahal','C de Grandhomme','UT Yadav','C de Grandhomme','Mohammed Siraj','TG Southee','Mohammed Siraj','TG Southee']
result1 = 1
res1 = match('rcb','mi', Batsmen1, Bowlers1, Batsmen2, Bowlers2,1)

Over Number  1
The Bowler is MJ McClenaghan
M Vohra Has scored 15.0

15.0 / 0
OVER STATS
runs: 15.0 , outs 0

Over Number  2
The Bowler is JJ Bumrah
Q de Kock Has scored 4.0

19.0 / 0
OVER STATS
runs: 4.0 , outs 0

Over Number  3
The Bowler is JP Duminy
M Vohra Has scored 6.0

25.0 / 0
OVER STATS
runs: 6.0 , outs 0

Over Number  4
The Bowler is MJ McClenaghan
Q de Kock got out
BB McCullum is the man next in

25.0 / 1
OVER STATS
runs: 0 , outs 1

Over Number  5
The Bowler is KH Pandya
BB McCullum Has scored 17.0

42.0 / 1
OVER STATS
runs: 17.0 , outs 0

Over Number  6
The Bowler is M Markande
M Vohra Has scored 3.0

45.0 / 1
OVER STATS
runs: 3.0 , outs 0

Over Number  7
The Bowler is KH Pandya
BB McCullum Has scored 17.0

62.0 / 1
OVER STATS
runs: 17.0 , outs 0

Over Number  8
The Bowler is M Markande
M Vohra Has scored 3.0

65.0 / 1
OVER STATS
runs: 3.0 , outs 0

Over Number  9
The Bowler is HH Pandya
BB McCullum Has scored 6.0

71.0 / 1
OVER STATS
runs: 6.0 , outs 0

Over Number  10
T

In [9]:
# Three matches:

# First match - Sunrisers Hyderabad vs Mumbai Indians (SRH vs MI)

# 1st Innings
# Batting - SRH
# Bowling - MI

srh_batting = ['Wriddhiman Saha','Shikar Dhawan',
               'Kane Williamson','Manish Pandey',
               'Shakib Al Hasan','Deepak Hooda',
               'Yusuf Pathan','Rashid Khan',
               'Siddarth Kaul','Sandeep Sharma','Billy Stanlake']

mi_bowling = ['Pradeep Sangwan','Jasprit Bumrah',
              'Pradeep Sangwan','Ben Cutting',
              'Krunal Pandya','Jasprit Bumrah',
              'Mayank Markhande','Mustafizur Rahman',
              'Mayank Markhande','Krunal Pandya',
              'Mayank Markhande','Mustafizur Rahman',
              'Mayank Markhande','Ben Cutting',
              'Jasprit Bumrah','Mustafizur Rahman',
              'Ben Cutting','Jasprit Bumrah',
              'Mustafizur Rahman','Ben Cutting']

# 2nd Innings
# Batting - MI
# Bowling - SRH

mi_batting = ['Rohit Sharma','Evin Lewis',
              'Ishan Kishan','Suryakumar Yadav',
              'Krunal Pandya','Kieron Pollard',
              'Ben Cutting','Pradeep Sangwan',
              'Mayank Markhande','Jasprit Bumrah','Mustafizur Rahman']

srh_bowling = ['Sandeep Sharma','Billy Stanlake',
               'Sandeep Sharma','Billy Stanlake',
               'Rashid Khan','Siddarth Kaul',
               'Shakib Al Hasan','Sandeep Sharma',
               'Shakib Al Hasan','Billy Stanlake',
               'Rashid Khan','Siddarth Kaul',
               'Rashid Khan','Shakib Al Hasan',
               'Billy Stanlake','Shakib Al Hasan',
               'Siddarth Kaul','Rashid Khan',
               'Sandeep Sharma','Siddarth Kaul']
# team_1n=input()
# team_2n=input()
result2 = 1
res2 = match('srh','mi', srh_batting, srh_bowling, mi_batting, mi_bowling,1)

Over Number  1
The Bowler is Jasprit Bumrah
Wriddhiman Saha Has scored 3.0

3.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  2
The Bowler is Pradeep Sangwan
Shikar Dhawan Has scored 3.0

6.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  3
The Bowler is Ben Cutting
Wriddhiman Saha Has scored 3.0

9.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  4
The Bowler is Krunal Pandya
Shikar Dhawan Has scored 3.0

12.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  5
The Bowler is Jasprit Bumrah
Wriddhiman Saha Has scored 3.0

15.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  6
The Bowler is Mayank Markhande
Shikar Dhawan Has scored 3.0

18.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  7
The Bowler is Mustafizur Rahman
Wriddhiman Saha Has scored 3.0

21.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  8
The Bowler is Mayank Markhande
Shikar Dhawan Has scored 3.0

24.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  9
The Bowler is Krunal Pandya
Wriddhiman Saha Has scored 3.0

27.0

In [10]:
# Second match - Kolkata Knight Riders vs Rajasthan Royals (KKR vs RR)

# 1st Innings
# Batting - RR
# Bowling - KKR

rr_batting = ['Rahul Tripathi','Jos Buttler',
              'Ajinkya Rahane','Sanju Samson',
              'Ben Stokes','Stuart Binny',
              'Krishanppa Gowtham','Jaydev Unadkat',
              'Ish Sodhi','Jofra Archer','Anureet Singh']

kkr_bowling = ['Shivam Mavi','Prasidh Krishna',
               'Shivam Mavi','Sunil Narine',
               'Andre Russell','Sunil Narine',
               'Andre Russell','Kuldeep Yadav',
               'Prasidh Krishna','Kuldeep Yadav',
               'Sunil Narine','Kuldeep Yadav',
               'Shivam Mavi','Kuldeep Yadav',
               'Shivam Mavi','Sunil Narine',
               'Prasidh Krishna','Andre Russell','Prasidh Krishna']

# 2nd Innings
# Batting - KKR
# Bowling - RR

kkr_batting = ['Sunil Narine','Chris Lynn',
               'Robin Uthappa','Nitish Rana',
               'Dinesh Karthik','Andre Russell',
               'Shubham Gill','Shivam Mavi',
               'Javon Searles','Prasidh Krishna','Kuldeep Yadav']

rr_bowling = ['Krishnappa Gowtham','Ben Stokes',
               'Jofra Archer','Ben Stokes',
               'Jofra Archer','Ben Stokes',
               'Ish Sodhi','Krishnappa Gowtham',
               'Ish Sodhi','Jaydev Unadkat',
               'Ish Sodhi','Jaydev Unadkat',
               'Anureet Singh','Jofra Archer',
               'Ish Sodhi','Ben Stokes',
               'Jaydev Unadkat','Jofra Archer']
# team_1n=input()
# team_2n=input()
result3 = -1
res3 = match('rr','kkr', rr_batting, rr_bowling, kkr_batting, kkr_bowling ,0 )

Over Number  1
The Bowler is Prasidh Krishna
Rahul Tripathi Has scored 3.0

3.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  2
The Bowler is Shivam Mavi
Jos Buttler Has scored 3.0

6.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  3
The Bowler is Sunil Narine
Rahul Tripathi Has scored 7.0

13.0 / 0
OVER STATS
runs: 7.0 , outs 0

Over Number  4
The Bowler is Andre Russell
Jos Buttler Has scored 18.0

31.0 / 0
OVER STATS
runs: 18.0 , outs 0

Over Number  5
The Bowler is Sunil Narine
Rahul Tripathi Has scored 7.0

38.0 / 0
OVER STATS
runs: 7.0 , outs 0

Over Number  6
The Bowler is Andre Russell
Jos Buttler Has scored 18.0

56.0 / 0
OVER STATS
runs: 18.0 , outs 0

Over Number  7
The Bowler is Kuldeep Yadav
Rahul Tripathi got out
Ajinkya Rahane is the man next in

56.0 / 1
OVER STATS
runs: 0 , outs 1

Over Number  8
The Bowler is Prasidh Krishna
Ajinkya Rahane Has scored 3.0

59.0 / 1
OVER STATS
runs: 3.0 , outs 0

Over Number  9
The Bowler is Kuldeep Yadav
Jos Buttler Has scored 6.0


In [11]:
# Third match - Royal Challengers Bangalore vs Delhi Daredevils (RCB vs DD)

# 1st Innings
# Batting - DD
# Bowling - RCB

dd_batting = ['Jason Roy','Gautham Gambhir',
              'Shreyas Iyer','Rishab Pant',
              'Glenn Maxwell','Rahul Tewatia',
              'Chris Morris','Vijay Shankar',
              'Shahbaz Nadeem','Harshal Patel','Trent Boult']

rcb_bowling = ['Umesh Yadav','Chris Woakes',
               'Umesh Yadav','Yuzvendra Chahal',
               'Umesh Yadav','Yuzvendra Chahal',
               'Mohammed Siraj','Washington Sundar',
               'Umesh Yadav','Washington Sundar',
               'Chris Woakes','Washington Sundar',
               'Mohammed Siraj','Washington Sundar',
               'Mohammed Siraj','Yuzvendra Chahal',
               'Chris Woakes','Mohammed Siraj',
               'Chris Woakes','Corey Anderson']

# 2nd Innings
# Batting - RCB
# Bowling - DD

rcb_batting = ['Quinton De Kock','Manan Vohra',
               'Virat Kohli','AB de Villiers',
               'Corey Anderson','Mandeep Singh',
               'Washington Sundar','Chris Woakes',
               'Umesh Yadav','Mohammed Siraj','Yuzvendra Chahal']

dd_bowling = ['Trent Boult','Glenn Maxwell',
              'Trent Boult','Glenn Maxwell',
              'Shahbaz Nadeem','Chris Morris',
              'Shahbaz Nadeem','Chris Morris',
              'Harshal Patel','Rahul Tewatia',
              'Harshal Patel','Rahul Tewatia',
              'Harshal Patel','Rahul Tewatia',
              'Chris Morris','Trent Boult',
              'Chris Morris','Trent Boult']
# team_1n=input()
# team_2n=input()
result4 = -1
res4 = match('dd','rcb', dd_batting, dd_bowling, rcb_batting, rcb_bowling ,1 )


Over Number  1
The Bowler is Chris Woakes
Jason Roy Has scored 3.0

3.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  2
The Bowler is Umesh Yadav
Gautham Gambhir Has scored 3.0

6.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  3
The Bowler is Yuzvendra Chahal
Jason Roy Has scored 3.0

9.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  4
The Bowler is Umesh Yadav
Gautham Gambhir Has scored 3.0

12.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  5
The Bowler is Yuzvendra Chahal
Jason Roy Has scored 3.0

15.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  6
The Bowler is Mohammed Siraj
Gautham Gambhir Has scored 3.0

18.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  7
The Bowler is Washington Sundar
Jason Roy Has scored 3.0

21.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  8
The Bowler is Umesh Yadav
Gautham Gambhir Has scored 3.0

24.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  9
The Bowler is Washington Sundar
Jason Roy Has scored 3.0

27.0 / 0
OVER STATS
runs: 3.0 

In [12]:
# Match 22

# Team1: 'KXIP'
# Team2: 'DD'

# Inn1:
kxipbat = ['KL Rahul','AJ Finch','MA Agarwal','KK Nair','Yuvraj Singh','DA Miller','R Ashwin','AJ Tye','BB Sran','AS Rajpoot','Mujeeb Ur Rahman']
ddbowl = ['TA Boult','Avesh Khan','TA Boult','Avesh Khan','LE Plunkett','Avesh Khan','DT Christian','LE Plunkett','DT Christian','A Mishra','GJ Maxwell','A Mishra','Avesh Khan','A Mishra','R Tewatia','A Mishra','LE Plunkett','DT Christian','LE Plunkett','TA Boult']
# Bowl = ['TA Boult','Avesh Khan','LE Plunkett','DT Christian','A Mishra','GJ Maxwell','R Tewatia']

# Inn2:
ddbat = ['PP Shaw','G Gambhir','GJ Maxwell','SS Iyer','RR Pant','DT Christian','R Tewatia','LE Plunkett', 'A Mishra','Avesh Khan','TA Boult']
# Bowl = ['AS Rajpoot','BB Sran','AJ Tye','R Ashwin','Mujeeb Ur Rahman']
kxipbowl = ['AS Rajpoot','BB Sran','AS Rajpoot','AJ Tye','AS Rajpoot','AJ Tye','AS Rajpoot','R Ashwin','Mujeeb Ur Rahman','R Ashwin','Mujeeb Ur Rahman','R Ashwin','BB Sran','R Ashwin','Mujeeb Ur Rahman','AJ Tye','BB Sran','AJ Tye','BB Sran','Mujeeb Ur Rahman']
print(len(kxipbat),len(ddbowl),len(ddbat), len(kxipbowl))
result5 = 1
res5 = match('kxip','dd', kxipbat, kxipbowl, ddbat, ddbowl , 1 )

11 20 11 20
Over Number  1
The Bowler is Avesh Khan
KL Rahul Has scored 3.0

3.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  2
The Bowler is TA Boult
AJ Finch Has scored 13.0

16.0 / 0
OVER STATS
runs: 13.0 , outs 0

Over Number  3
The Bowler is Avesh Khan
KL Rahul Has scored 3.0

19.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  4
The Bowler is LE Plunkett
AJ Finch Has scored 3.0

22.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  5
The Bowler is Avesh Khan
KL Rahul Has scored 3.0

25.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  6
The Bowler is DT Christian
AJ Finch Has scored 7.0

32.0 / 0
OVER STATS
runs: 7.0 , outs 0

Over Number  7
The Bowler is LE Plunkett
KL Rahul Has scored 3.0

35.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  8
The Bowler is DT Christian
AJ Finch Has scored 7.0

42.0 / 0
OVER STATS
runs: 7.0 , outs 0

Over Number  9
The Bowler is A Mishra
KL Rahul Has scored 5.0

47.0 / 0
OVER STATS
runs: 5.0 , outs 0

Over Number  10
The Bowler is GJ Maxwell

In [13]:
#Match 25

# Team1: 'SRH'
# Team2: 'KXIP'

# Inn1:
Bat1 = ['S Dhawan','KS Williamson','WP Saha','MK Pandey','Shakib Al Hasan','YK Pathan','Mohammad Nabi','Rashid Khan','Basil Thampi','S Kaul','Sandeep Sharma']
# Bowl2 = ['AS Rajpoot','BB Sran','R Ashwin','MK Tiwary','Mujeeb Ur Rahman','AJ Tye']
Bowl2 = ['AS Rajpoot','BB Sran','AS Rajpoot','BB Sran','AS Rajpoot','BB Sran','R Ashwin','MK Tiwary','R Ashwin','Mujeeb Ur Rahman','AJ Tye','Mujeeb Ur Rahman','AJ Tye','Mujeeb Ur Rahman','R Ashwin','Mujeeb Ur Rahman','AJ Tye','R Ashwin','AJ Tye','AS Rajpoot']

# Inn2
Bat2 = ['KL Rahul','CH Gayle','MA Agarwal','KK Nair','AJ Finch','MK Tiwary','R Ashwin','AJ Tye','BB Sran','AS Rajpoot','Mujeeb Ur Rahman']
# Bowl1 = ['Sandeep Sharma','Mohammad Nabi','S Kaul','Basil Thampi','Rashid Khan','Shakib Al Hasan']
Bowl1 = ['Sandeep Sharma','Mohammad Nabi','Sandeep Sharma','Mohammad Nabi','Sandeep Sharma','S Kaul','Basil Thampi','Rashid Khan','Basil Thampi','Shakib Al Hasan','S Kaul','Rashid Khan','Shakib Al Hasan','Rashid Khan','Shakib Al Hasan','Sandeep Sharma','S Kaul','Rashid Khan','S Kaul','Basil Thampi']
result6 = 1
res6 = match('srh','kxip',Bat1,Bowl1,Bat2,Bowl2, 1)

Over Number  1
The Bowler is BB Sran
S Dhawan Has scored 10.0

10.0 / 0
OVER STATS
runs: 10.0 , outs 0

Over Number  2
The Bowler is AS Rajpoot
KS Williamson Has scored 8.0

18.0 / 0
OVER STATS
runs: 8.0 , outs 0

Over Number  3
The Bowler is BB Sran
S Dhawan Has scored 10.0

28.0 / 0
OVER STATS
runs: 10.0 , outs 0

Over Number  4
The Bowler is AS Rajpoot
KS Williamson Has scored 8.0

36.0 / 0
OVER STATS
runs: 8.0 , outs 0

Over Number  5
The Bowler is BB Sran
S Dhawan Has scored 10.0

46.0 / 0
OVER STATS
runs: 10.0 , outs 0

Over Number  6
The Bowler is R Ashwin
KS Williamson Has scored 4.0

50.0 / 0
OVER STATS
runs: 4.0 , outs 0

Over Number  7
The Bowler is MK Tiwary
S Dhawan Has scored 9.0

59.0 / 0
OVER STATS
runs: 9.0 , outs 0

Over Number  8
The Bowler is R Ashwin
KS Williamson Has scored 4.0

63.0 / 0
OVER STATS
runs: 4.0 , outs 0

Over Number  9
The Bowler is Mujeeb Ur Rahman
S Dhawan Has scored 5.0

68.0 / 0
OVER STATS
runs: 5.0 , outs 0

Over Number  10
The Bowler is AJ Tye


In [14]:
# Qualifier1:

# Team1 = 'SRH'
# Team2 = 'CSK'

# Inn2

Bat1= ['SR Watson','F du Plessis','SK Raina','AT Rayudu','MS Dhoni','DJ Bravo','RA Jadeja','DL Chahar','Harbhajan Singh','SN Thakur','L Ngidi']
# Bowl = ['B Kumar','Sandeep Sharma','S Kaul','CR Brathwaite','Rashid Khan','Shakib Al Hasan']
Bowl2 = ['B Kumar','Sandeep Sharma','B Kumar','S Kaul','B Kumar','S Kaul','CR Brathwaite','Rashid Khan','CR Brathwaite','Rashid Khan','Shakib Al Hasan','Rashid Khan','Sandeep Sharma','Shakib Al Hasan','Sandeep Sharma','Rashid Khan','S Kaul','CR Brathwaite','S Kaul','B Kumar']

# Inn1

Bat2 = ['S Dhawan','SP Goswami','KS Williamson','MK Pandey','Shakib Al Hasan','YK Pathan','CR Brathwaite','B Kumar','Rashid Khan','S Kaul','Sandeep Sharma']
# Bowl = ['DL Chahar','L Ngidi','SN Thakur','DJ Bravo','RA Jadeja']
Bowl1 = ['DL Chahar','L Ngidi','DL Chahar','L Ngidi','SN Thakur','DL Chahar','DJ Bravo','RA Jadeja','DJ Bravo','RA Jadeja','DL Chahar','RA Jadeja','SN Thakur','RA Jadeja','DJ Bravo','L Ngidi','DJ Bravo','SN Thakur','L Ngidi','SN Thakur']
result7 = -1
res7 = match('srh','csk',Bat1,Bowl1,Bat2,Bowl2,0)

Over Number  1
The Bowler is Sandeep Sharma
SR Watson Has scored 8.0

8.0 / 0
OVER STATS
runs: 8.0 , outs 0

Over Number  2
The Bowler is B Kumar
F du Plessis Has scored 9.0

17.0 / 0
OVER STATS
runs: 9.0 , outs 0

Over Number  3
The Bowler is S Kaul
SR Watson Has scored 0.0

17.0 / 0
OVER STATS
runs: 0.0 , outs 0

Over Number  4
The Bowler is B Kumar
F du Plessis Has scored 9.0

26.0 / 0
OVER STATS
runs: 9.0 , outs 0

Over Number  5
The Bowler is S Kaul
SR Watson Has scored 0.0

26.0 / 0
OVER STATS
runs: 0.0 , outs 0

Over Number  6
The Bowler is CR Brathwaite
F du Plessis Has scored 10.0

36.0 / 0
OVER STATS
runs: 10.0 , outs 0

Over Number  7
The Bowler is Rashid Khan
SR Watson Has scored 12.0

48.0 / 0
OVER STATS
runs: 12.0 , outs 0

Over Number  8
The Bowler is CR Brathwaite
F du Plessis Has scored 10.0

58.0 / 0
OVER STATS
runs: 10.0 , outs 0

Over Number  9
The Bowler is Rashid Khan
SR Watson Has scored 12.0

70.0 / 0
OVER STATS
runs: 12.0 , outs 0

Over Number  10
The Bowler is

In [15]:
print((result1==res1),(result2==res2),(result3==res3),(result4==res4),(result5==res5),(result6==res6),(result7==res7))

True True False False True True False


In [16]:
# Delhi daredevis vs Kolkata Knight Riders

# 1st innings 
# Batting - DD
# Bowling - KKR

dd_batting = ['PP Shaw','Colin Munro',
				'Shreyas Iyer','Rishabh Pant',
				'Glenn Maxwell','Vijay Shankar',
				'Rahul Tewatia','Liam Plunkett',
				'Amit Mishra','Avesh Khan','Trent Boult']

kkr_bowling = ['Piyush Chawla','Kuldeep Yadav',
			   'Piyush Chawla','Shivam Mavi',
			   'Sunil Narine','Mitchell Johnson',
			   'Shivam Mavi','Andre Russell',
			   'Mitchell Johnson','Sunil Narine',
			   'Andre Russell','Kuldeep Yadav',
			   'Mitchell Johnson','Piyush Chawla',
			   'Andre Russell','Piyush Chawla',
			   'Shivam Mavi','Sunil Narine',
			   'Mitchell Johnson','Shivam Mavi']

# 2st innings 
# Batting - KKR
# Bowling - DD

kkr_batting = ['Chris Lynn','Sunil Narine',
				'Robin Uthappa','Nitish Rana',
				'Dinesh Karthik','Shubman Gill',
				'Andre Russell','Shivam Mavi',
				'Piyush Chawla','Mitchell Johnson','Kuldeep Yadav']

dd_bowling = ['Trent Boult','Glenn Maxwell',
			  'Trent Boult','Avesh Khan',
			  'Liam Plunkett','Avesh Khan',
			  'Liam Plunkett','Amit Mishra',
			  'Vijay Shankar','Amit Mishra',
			  'Liam Plunkett','Avesh Khan',
			  'Trent Boult','Amit Mishra',
			  'Rahul Tewatia','Amit Mishra'
			  'Liam Plunkett','Avesh Khan'
			  'Glenn Maxwell','Trent Boult']
resultfin = 1 
resfin = match('dd','kkr',dd_batting,dd_bowling,kkr_batting,kkr_bowling,1)

Over Number  1
The Bowler is Kuldeep Yadav
PP Shaw Has scored 3.0

3.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  2
The Bowler is Piyush Chawla
Colin Munro Has scored 3.0

6.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  3
The Bowler is Shivam Mavi
PP Shaw Has scored 3.0

9.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  4
The Bowler is Sunil Narine
Colin Munro Has scored 3.0

12.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  5
The Bowler is Mitchell Johnson
PP Shaw Has scored 3.0

15.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  6
The Bowler is Shivam Mavi
Colin Munro Has scored 3.0

18.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  7
The Bowler is Andre Russell
PP Shaw Has scored 3.0

21.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  8
The Bowler is Mitchell Johnson
Colin Munro Has scored 3.0

24.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  9
The Bowler is Sunil Narine
PP Shaw Has scored 3.0

27.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  10
The Bowl

In [17]:
resfin == resultfin

True

In [18]:
#Match 1 - SRH vs CSK
#may 22 2018 Csk won

#1st innings - SRH batting

srh_bat = ['Shikar Dhawan','Shreevats Goswami', 'Kane Williamson','Manish Pandey', 'Shakib Al Hasan','Yusuf Pathan',
          'Carlos Brathwaite','Bhuvneshwar Kumar','Rashid Khan' , 'Siddarth Kaul' , 'Sandeep Sharma']

csk_bowl = ['Deepak Chahar','Lungi Ngidi' , 'Deepak Chahar','Lungi Ngidi' ,
            'Shardul Thakur' ,'Deepak Chahar','Dwayne Bravo','Ravindra Jadeja',
            'Dwayne Bravo', 'Ravindra Jadeja','Deepak Chahar' ,'Ravindra Jadeja',
            'Shardul Thakur','Ravindra Jadeja', 'Dwayne Bravo','Lungi Ngidi' ,
            'Dwayne Bravo','Shardul Thakur' ,'Lungi Ngidi' ,'Shardul Thakur'
           ]

#2nd innings- Csk batting

csk_bat = ['Shane Watson' , 'Faf du Plessis' , 'Suresh Raina', 'Ambati Rayudu','MS Dhoni','Dwayne Bravo','Ravindra Jadeja',
           'Deepak Chahar','Harbhajan Singh','Shardul Thakur',
          'Lungi Ngidi']

srh_bowl = ['Bhuvneshwar Kumar','Sandeep Sharma','Bhuvneshwar Kumar','Siddarth Kaul',
            'Bhuvneshwar Kumar','Siddarth Kaul','Carlos Brathwaite','Rashid Khan',
            'Carlos Brathwaite', 'Rashid Khan', 'Shakib Al Hasan', 'Rashid Khan',
            'Sandeep Sharma','Shakib Al Hasan','Sandeep Sharma','Rashid Khan',
            'Siddarth Kaul','Carlos Brathwaite','Siddarth Kaul','Bhuvneshwar Kumar'
           ]
result = -1
predresult = match('srh','csk',srh_bat,srh_bowl,csk_bat,csk_bowl,1)
result == predresult

Over Number  1
The Bowler is Lungi Ngidi
Shikar Dhawan Has scored 3.0

3.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  2
The Bowler is Deepak Chahar
Shreevats Goswami Has scored 3.0

6.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  3
The Bowler is Lungi Ngidi
Shikar Dhawan Has scored 3.0

9.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  4
The Bowler is Shardul Thakur
Shreevats Goswami Has scored 3.0

12.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  5
The Bowler is Deepak Chahar
Shikar Dhawan Has scored 3.0

15.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  6
The Bowler is Dwayne Bravo
Shreevats Goswami Has scored 3.0

18.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  7
The Bowler is Ravindra Jadeja
Shikar Dhawan Has scored 3.0

21.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  8
The Bowler is Dwayne Bravo
Shreevats Goswami Has scored 3.0

24.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  9
The Bowler is Ravindra Jadeja
Shikar Dhawan Has scored 3.0

27.0 / 0
OV

False

In [19]:
#match 2 KXIP vs CSK
# may 20th 2018 csk won

#1st innings - kxip batting


kxip_bat = ['Lokesh Rahul', 'Chris Gayle' , 'Aaron Finch' , 'Manoj Tiwary' , 'David Miller' ,'Karun Nair' , 'Axar Patel',
           'Andrew Tye' , 'Mohit Sharma', 'Ankit Rajpoot','']

csk_bowl = ['Deepak Chahar','Lungi Ngidi','Deepak Chahar','Lungi Ngidi',
               'Deepak Chahar','Lungi Ngidi','Deepak Chahar','Harbhajan Singh', 
               'Shardul Thakur', 'Dwayne Bravo','Shardul Thakur','Ravindra Jadeja',
               'Dwayne Bravo','Ravindra Jadeja','Dwayne Bravo', 'Ravindra Jadeja',
               'Shardul Thakur','Lungi Ngidi','Dwayne Bravo','Shardul Thakur']
               

#2nd innings - csk batting

csk_bat = ['Ambati Rayudu' , 'Faf du Plessis' , 'Suresh Raina'  , 'Sam Billings', 'Harbhajan Singh' ,'Deepak Chahar',
           'MS Dhoni','Dwayne Bravo' , 'Ravindra Jadeja' , 'Shardul Thakur' , 'Lungi Ngidi']


kxip_bowl = ['Ankit Rajpoot','Mohit Sharma','Ankit Rajpoot','Andrew Tye',
             'Ankit Rajpoot', 'Mohit Sharma','Ankit Rajpoot','Axar Patel',
             'Andrew Tye','Axar Patel','Ravichandran Ashwin','Andrew Tye',
             'Ravichandran Ashwin', 'Axar Patel','Ravichandran Ashwin' ,'Axar Patel',
             'Ravichandran Ashwin','Mohit Sharma','Andrew Tye', 'Mohit Sharma']
            
result = -1
predresult = match('kxip','csk',kxip_bat,kxip_bowl,csk_bat,csk_bowl,1)
result == predresult



Over Number  1
The Bowler is Lungi Ngidi
Lokesh Rahul Has scored 3.0

3.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  2
The Bowler is Deepak Chahar
Chris Gayle Has scored 8.0

11.0 / 0
OVER STATS
runs: 8.0 , outs 0

Over Number  3
The Bowler is Lungi Ngidi
Lokesh Rahul Has scored 3.0

14.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  4
The Bowler is Deepak Chahar
Chris Gayle Has scored 8.0

22.0 / 0
OVER STATS
runs: 8.0 , outs 0

Over Number  5
The Bowler is Lungi Ngidi
Lokesh Rahul Has scored 3.0

25.0 / 0
OVER STATS
runs: 3.0 , outs 0

Over Number  6
The Bowler is Deepak Chahar
Chris Gayle Has scored 8.0

33.0 / 0
OVER STATS
runs: 8.0 , outs 0

Over Number  7
The Bowler is Harbhajan Singh
Lokesh Rahul Has scored 4.0

37.0 / 0
OVER STATS
runs: 4.0 , outs 0

Over Number  8
The Bowler is Shardul Thakur
Chris Gayle Has scored 14.0

51.0 / 0
OVER STATS
runs: 14.0 , outs 0

Over Number  9
The Bowler is Dwayne Bravo
Lokesh Rahul Has scored 7.0

58.0 / 0
OVER STATS
runs: 7.0 , outs 0


False

In [20]:
sc.stop()